In [ ]:
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt 
import numpy as np



import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_absolute_percentage_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
from xgboost import XGBRegressor
from ydata_profiling import ProfileReport


from scalecast.Forecaster import Forecaster
alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

In [2]:
import optuna

## Import Data

In [29]:
train = pd.read_csv('train.csv')
blind = pd.read_csv('test.csv')

In [6]:
data_profile = ProfileReport(train)

In [35]:
def feature_eng(df):
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])

    # Extract date features
    df['day_of_year'] = df['date'].dt.dayofyear
    df['day_week'] = df['date'].dt.dayofweek
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year

    # Encode categorical features
    le = LabelEncoder()
    df['country'] = le.fit_transform(df['country'])
    df['store'] = le.fit_transform(df['store'])
    df['product'] = le.fit_transform(df['product'])

    # Drop missing values and irrelevant columns
    df = df.dropna()
    df = df.set_index('date')
    df = df.drop(columns=['id'])  # Dropping columns that are not useful for modeling
    # Separate features and target
    try: 
        X = df.drop(columns=['num_sold'])
    except:
        X=df
    try:
        y = df['num_sold']
    except:
        y = None
    # Scale features
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

    return df, X,  y

# Feature engineering
df, X, y = feature_eng(train)

# Splitting the data
SPLIT = 0.85
split_index = int(SPLIT * len(X))

X_train = X[:split_index]
y_train = y[:split_index]
X_test = X[split_index:]
y_test = y[split_index:]

# Verify shapes
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (188070, 7)
y_train shape: (188070,)
X_test shape: (33189, 7)
y_test shape: (33189,)


In [37]:
blind, X_blind, y_blind = feature_eng(blind)

In [40]:
def objective(trial):
    print(f"🔄 Starting Trial {trial.number + 1} out of {20}")  # Replace 20 with your actual n_trials

    # Define hyperparameters
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 200),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "max_depth": trial.suggest_int("max_depth", 3, 18),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.1, 10.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.1, 10.0),
        "device": "cuda",  # Ensure GPU usage
        "tree_method": "hist"  # Recommended for GPU acceleration in XGBoost 2.0+
    }

    # Train the model
    model = xgb.XGBRegressor(**param)
    model.fit(X_train, y_train)

    # Predict & compute MAPE
    preds = model.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, preds)
    blind_preds = model.predict(blind)
    #blind_mape = mean_absolute_percentage_error(y_train, blind_preds)
    print(f"✅ Completed Trial {trial.number + 1} - MAPE: {mape:.4f}")

    return mape

# Run Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)  # ✅ Use explicit n_trials here


[I 2025-01-29 17:40:33,354] A new study created in memory with name: no-name-841d673c-ffd6-4b29-826b-39a155aabe20


🔄 Starting Trial 1 out of 20


In [21]:
test = pd.read_csv('test.csv')

In [23]:
blind_predition_model = model.fit(test)

NameError: name 'model' is not defined